In [26]:
from datasets import load_dataset
from pathlib import Path

In [69]:
base_path = Path.home() / '.cache' / 'xmen' / 'symptemist'

In [14]:
data = load_dataset(
    path="../../biomedical/bigbio/hub/hub_repos/symptemist/symptemist.py", 
    name="symptemist_linking_bigbio_kb"
)
data

Found cached dataset symptemist (/dhc/home/florian.borchert/.cache/huggingface/datasets/symptemist/symptemist_linking_bigbio_kb/2.0.0/b6cad1bf9c574bfc3ac6a1ea50a3b8ff5b30de3ee0a2b94b4d692546ddbf011f)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 304
    })
})

In [67]:
from xmen.linkers import default_ensemble

In [70]:
linker = default_ensemble(base_path / 'index')

[09/15/23 21:58:04] INFO     Loading hierarchical faiss   ]8;id=394137;file:///dhc/home/florian.borchert/workspace/xmen/xmen/linkers/sap_bert_linker.py\sap_bert_linker.py]8;;\:]8;id=394764;file:///dhc/home/florian.borchert/workspace/xmen/xmen/linkers/sap_bert_linker.py#153\153]8;;\
                             index                                              
                    INFO     Loading index from              ]8;id=118718;file:///dhc/home/florian.borchert/workspace/xmen/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=892162;file:///dhc/home/florian.borchert/workspace/xmen/xmen/linkers/faiss_indexer.py#64\64]8;;\
                             /dhc/home/florian.borchert/.cac                    
                             he/xmen/symptemist/index/sapber                    
                             t/embed_faiss_hier.pickle                          
[09/15/23 21:58:12] INFO     Loaded index of type <class     ]8;id=197716;file:///dhc/home/florian

In [71]:
from xmen.data import Sampler

In [102]:
sample = data#Sampler(52, 30).transform_batch(data)

In [103]:
from xmen.data import get_cuis
from xmen import load_kb

In [104]:
candidates = linker.predict_batch(sample, batch_size=128)

Map:   0%|          | 0/304 [00:00<?, ? examples/s]

In [105]:
cn = linker.linkers_fn['ngram']().predict_batch(sample['train'])

Map:   0%|          | 0/304 [00:00<?, ? examples/s]

In [106]:
cs = linker.linkers_fn['sapbert']().predict_batch(sample['train'])

Map:   0%|          | 0/304 [00:00<?, ? examples/s]

In [107]:
from xmen.evaluation import evaluate_at_k

In [108]:
_ = evaluate_at_k(sample['train'], cn)

Perf@1 0.3289027149321267
Perf@2 0.43721719457013575
Perf@4 0.5
Perf@8 0.5378959276018099
Perf@16 0.5808823529411765
Perf@32 0.6233031674208145
Perf@64 0.6504524886877828


In [109]:
_ = evaluate_at_k(sample['train'], cs)

Perf@1 0.40384615384615385
Perf@2 0.520079185520362
Perf@4 0.5871040723981901
Perf@8 0.641685520361991
Perf@16 0.681843891402715
Perf@32 0.7149321266968326
Perf@64 0.7423642533936652


In [110]:
_ = evaluate_at_k(sample['train'], candidates['train'])

Perf@1 0.43240950226244346
Perf@2 0.5313914027149321
Perf@4 0.5947398190045249
Perf@8 0.6464932126696833
Perf@16 0.6894796380090498
Perf@32 0.7214366515837104
Perf@64 0.7474547511312217


In [79]:
cuis = get_cuis(data['train'])

In [80]:
len(cuis)

3536

In [30]:
kb = load_kb(base_path / 'symptemist.jsonl')

In [ ]:
gaz = pd.read_csv('../data/symptemist_gazetter_snomed_ES_v1.tsv', sep='\t')

In [ ]:
gaz[gaz.code == 309306009]

,code,language,term,semantic_tag,mainterm


In [ ]:
dat = pd.read_csv('../data/symptemist_tsv_train_subtask2.tsv', sep='\t')

In [ ]:
idx = set(gaz.code.values.astype('str'))

In [ ]:
dat[~dat.code.isin(idx)]

,filename,label,start_span,end_span,text,code,sem_rel,is_abbrev,is_composite,need_context
37,es-S0376-78922015000200010-1,SINTOMA,1139,1160,actividad paroxística,1.56255710001191E+016,EXACT,False,False,False
50,es-S0376-78922009000300010-1,SINTOMA,1538,1587,adelgazamiento por desgrasamiento de los colgajos,1202017008,EXACT,False,False,True
176,es-S0376-78922015000400013-1,SINTOMA,243,314,anfractuosidades en el tejido cutáneo-adiposo ...,1.06828710001191E+016,NARROW,False,False,True
225,es-S1130-01082009000700012-1,SINTOMA,1006,1024,apéndice epiploico,1.47202310001191E+016,EXACT,False,False,True
236,es-S0004-06142008000600014-1,SINTOMA,687,702,áreas quísticas,1.62173410001191E+016,EXACT,False,False,False
...,...,...,...,...,...,...,...,...,...,...
3285,es-S0212-71992006000800007-1,SINTOMA,4427,4449,tinción de ZN negativo,309306009,NARROW,True,False,False
3288,es-S1130-01082008001000010-1,SINTOMA,1988,2010,tinción Zielh negativa,309306009,NARROW,False,False,False
3289,es-S0212-71992004000800009-1,SINTOMA,2878,2959,tinciones de muestras respiratorias fueron neg...,309306009,NARROW,False,False,False
3491,es-S1130-05582003000600004-1,SINTOMA,426,480,zona anterior izquierda mandibular se apreció ...,1.06793510001191E+016,NARROW,False,False,False


In [31]:
[c for c in cuis if not c in kb.cui_to_entity]

['NO_CODE',
 '309306009',
 '271594007',
 '257552002',
 '14760008',
 '161980004',
 '271823003',
 'NO_CODE',
 'NO_CODE',
 'NO_CODE',
 'NO_CODE',
 '309306009',
 '248239003',
 '271823003',
 'NO_CODE',
 '165232002',
 'NO_CODE',
 'NO_CODE',
 '274643008',
 '193982009',
 'NO_CODE',
 '267038008',
 '1.57499610001191E',
 '016',
 'NO_CODE',
 'NO_CODE',
 '271782001',
 '271823003',
 'NO_CODE',
 '271594007',
 '168265009',
 'NO_CODE',
 '309306009',
 '131148009',
 '268926000',
 '1.56255710001191E',
 '016',
 'NO_CODE',
 'NO_CODE',
 'NO_CODE',
 '168265009',
 '251435008',
 '267038008',
 '309306009',
 '1141974008',
 '309306009',
 '703630003',
 '221360009',
 '1202017008',
 'NO_CODE',
 '309306009',
 '309306009',
 '271823003',
 '1.06787910001191E',
 '016',
 'NO_CODE',
 '131148009',
 'NO_CODE',
 '251384004',
 '239551005',
 '309306009',
 'NO_CODE',
 '1.62173410001191E',
 '016',
 '274718005',
 'NO_CODE',
 'NO_CODE',
 '309305008',
 'NO_CODE',
 'NO_CODE',
 '274643008',
 'NO_CODE',
 '131148009',
 '309569001',
 '267